# Test of GAT
- use DGL

In [26]:
import dgl
import json
import torch
import torch as th
from tqdm import tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [27]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")

print(device)


cuda:3


## Data Loader

In [28]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data = self.data_list[idx]
        
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)
        g = dgl.add_self_loop(g) # this would cause the num_edges = 2+3 in each sub_graph
        
        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        
        return g, th.tensor(data["y"]).to(self.device)
    
#     def __getitem__(self, idx):
#         data = self.data_list[idx]

#         g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)
#         g = dgl.add_self_loop(g) 

#         g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
#         g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

#         return g, th.tensor(data["y"]).to(self.device)



def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [30]:
datasets = ['train', 'valid', 'test']
dataloaders = {}

for dataset_name in tqdm(datasets):
    file_path = f"../data/final_small_version/remaining_{dataset_name}.jsonl"
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in f]
    
    dataset = GraphDataset(data_list, device)
    dataloaders[dataset_name] = DataLoader(dataset, batch_size=32, shuffle=False, collate_fn=collate)
    
print("Done!")

  0%|          | 0/3 [00:00<?, ?it/s]

../data/training_data/repeated_train.jsonl


 33%|███▎      | 1/3 [00:10<00:21, 10.84s/it]

../data/training_data/repeated_valid.jsonl


 67%|██████▋   | 2/3 [00:14<00:06,  6.68s/it]

../data/training_data/repeated_test.jsonl


100%|██████████| 3/3 [00:18<00:00,  6.11s/it]

Done!


### Model

In [31]:
class GAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, dropout_prob=0.2):
        super(GAT, self).__init__()
        
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads=num_heads, activation=F.relu)
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, num_heads=num_heads)
        
        # Adding Batch Normalization after each GAT layer
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim * num_heads)
        self.batchnorm2 = nn.BatchNorm1d(out_dim)
        
        # Adding Dropout for regularization
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, g, h):
        # Apply GAT layers
        h = self.layer1(g, h)
        h = h.view(h.shape[0], -1)
        
#         h = self.batchnorm1(h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h).squeeze(1)
        
        # Apply Batch Normalization after second GAT layer
#         h = self.batchnorm2(h)
        
        # Store the output as a new node feature
        g.ndata['h_out'] = h

        # Use mean pooling to aggregate this new node feature
        h_agg = dgl.mean_nodes(g, feat='h_out')
        return h_agg


- Model Forward  

In [32]:
def model_fn(data, model, criterion, device, count=1):
    """Forward a batch through the model."""
    batched_g, labels = data
#     print(batch_g)
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
    logits = logits.mean(dim=1)
#     print(logits)
    
    loss = criterion(logits, labels)
#     print(batched_g.ndata['feat'].dtype)
#     print("Logits shape:", logits.shape)  # Expected: (batch_size, 168)
#     print("Labels shape:", labels.shape)  # Expected: (batch_size)

    # Get the class id with the highest probability.
    preds = logits.argmax(1)
    if count%100==0: print(f"This is {count}-th prediction metrix: {preds}")
#     print(preds)
    
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

'''
batched_g is like: 
Graph(num_nodes=96, num_edges=160, ndata_schemes={'feat': Scheme(shape=(1,), dtype=torch.int64)}, edata_schemes={})
num_nodes = 3*batch_size, num_edges = 5*batch_size
labels is like: tensor([ 76,   0,   0,   0,   0,   0,   0,   0,   0,  76,   0,  76,   0,   0,
                          0,   0,  76,   0,  30,  92,   0,   0,  76,   0,   0,   0,   0,   0,
                        116,   0,  76,  76])
'''

"\nbatched_g is like: \nGraph(num_nodes=96, num_edges=160, ndata_schemes={'feat': Scheme(shape=(1,), dtype=torch.int64)}, edata_schemes={})\nnum_nodes = 3*batch_size, num_edges = 5*batch_size\nlabels is like: tensor([ 76,   0,   0,   0,   0,   0,   0,   0,   0,  76,   0,  76,   0,   0,\n                          0,   0,  76,   0,  30,  92,   0,   0,  76,   0,   0,   0,   0,   0,\n                        116,   0,  76,  76])\n"

### Training

In [33]:
model = GAT(in_dim=1, hidden_dim=16, out_dim=168, num_heads=8)
# in_dim means the dimension of the node_feat(1 dim, since the design of our dataset), if a node has multiple feature -> in_dim > 1
# out_dim means the # of the categories -> 168 for out tasks


model = model.to(device)
print(f"Using: {device} now")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=20, num_training_steps=total_steps)

criterion = nn.CrossEntropyLoss()
total_steps = 100


for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    count = 0 
    
    for data in dataloaders['train']:
        print(data)
        count += 1
        
#         print(data[0])
        loss, accuracy = model_fn(data, model, criterion, device, count)
#         print('hi')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()
        num_batches += 1
        
    scheduler.step()
    print(f"total count: {count}")
    
    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    print(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    # Validate
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for batched_g in dataloaders['valid']:
            loss, accuracy = model_fn(batched_g, model, criterion, device)
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    avg_loss = total_loss / num_batches
    print(f'Validation Loss: {avg_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}')


    # Save checkpoint
    if epoch%20 == 0:
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f"../checkpoint_GAT/checkpoint_{epoch}.pt")
    

# After all epochs
model.eval()
total = 0
correct = 0
with torch.no_grad():
#     for batched_g, labels in test_dataloader:
    for batched_g, labels in dataloaders['test']:
        batched_g = batched_g.to(device)
        labels = labels.to(device)
        logits = model(batched_g, batched_g.ndata['feat'])
        _, predicted = torch.max(logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy: %d %%' % (100 * correct / total))

Using: cuda:3 now


  0%|          | 0/100 [00:00<?, ?it/s]


DGLError: Expect number of features to match number of edges. Got 1 and 3 instead.

In [ ]:
# 1. 定義圖神經網路模型
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size)
        self.conv2 = GraphConv(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        # 將每個圖的所有節點特徵進行平均
        hg = dgl.mean_nodes(g, 'h')
        return hg

# 2. 定義數據集
class GraphDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data = self.data_list[idx]
        g = dgl.graph((data["edge_index"][0], data["edge_index"][1]), num_nodes=data["num_nodes"])
        g = dgl.add_self_loop(g)  # add self loop to each node
        g.ndata['feat'] = torch.tensor(data["node_feat"])
        # g.edata['feat'] = torch.tensor(data["edge_attr"])
        return g, torch.tensor(data["y"])


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


with open("../data/final_small_version/remaining_train.jsonl") as f:
    train_data_list = [json.loads(line) for line in f]
train_dataset = GraphDataset(train_data_list)
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=dgl.batch)

with open("../data/final_small_version/remaining_valid.jsonl") as f:
    val_data_list = [json.loads(line) for line in f]
val_dataset = GraphDataset(val_data_list)
# val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=dgl.batch)

with open("../data/final_small_version/remaining_test.jsonl") as f:
    test_data_list = [json.loads(line) for line in f]
test_dataset = GraphDataset(test_data_list)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=dgl.batch)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=collate)



# 4. 創建模型並訓練
model = GCN(1, 16, 168) # 1是輸入特徵的維度，16是隱藏層大小，168是類別數量
# model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in tqdm(range(100)):
    for batched_g, labels in train_dataloader:
        # batched_g, labels = batched_g.to(device), labels.to(device)
        logits = model(batched_g, batched_g.ndata['feat'])
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))

    # 儲存 checkpoint
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f"./checkpoint_GAT/checkpoint_{epoch}.pt")

# 5. 驗證模型
model.eval()
with torch.no_grad():
    for batched_g, labels in val_dataloader:
        logits = model(batched_g, batched_g.ndata['feat'])
        _, predicted = torch.max(logits.data, 1)
        total = labels.size(0)
        correct = (predicted == labels).sum().item()
        print('Accuracy: %d %%' % (100 * correct / total))